# Photonic circuit simulations

At the scale of an entire chip, full electromagnetic simulations become prohibitive. To gain insight on the behaviour of an entire circuit, circuit simulations using **compact models** can instead be performed.

In the semiconductor industry, a compact model is a "computationally efficient description of the terminal properties of a device as a function of terminal voltages" [(Sivakumar Mudanai (Intel), on Nanohub)](https://nanohub.org/resources/21459/download/2014.06.10-Mudanai-NEEDS.pdf). 

We can generalize the concept to photonics : a compact model would be an efficient description of the properties of a device as a function of a few parameters such as wavelength, voltage applied, etc. The properties we are most often interested in are the S-parameters, the elements of the scattering matrix. The concept originated in microwave design, so a good introduction is contained in Ch. 4.3 of [1]. As usual, the main reference for the class [2] has a good chapter on it (Ch. 9).

1. Pozar, D. M. (1990). Microwave Engineering
2. Chrostowski, L., & Hochberg, M. (2015). Photonic circuit modelling. In Silicon Photonics Design: From Devices to Systems (pp. 313-348). Cambridge: Cambridge University Press.

# S-parameters



# What are your options?

## Commercial

Some I've used, but there are a lot more :
[Lumerical](https://www.lumerical.com/), [Synopsys](https://www.synopsys.com/photonic-solutions.html), . These may be easier to use in a "production" environment, and have more support.

## Why open-source (MEEP/MPB)?

* Free!
* Flexible. Good for research.
* Transparent. Good for teaching.
* Widely-used

<img src="images/MEEP.png" width="40%">

<img src="images/MPB.PNG" width="40%">

 # What is PyMEEP?

It is the Python wrapper to MEEP and MPB. These tools can also be used from a Scheme interface. 

* **MPB : MIT Periodic Bands**
    * "MPB computes definite-frequency eigenstates, or harmonic modes, of Maxwell's equations in periodic dielectric structures for arbitrary wavevectors, using fully-vectorial and three-dimensional methods. It is applicable to many problems in optics, such as waveguides and resonator systems, and photonic crystals. [1]
    

* **MEEP : MIT Electromagnetic Equation Propagation**
    * "A time-domain electromagnetic simulation simply evolves Maxwell's equations over time within some finite computational volume, essentially performing a kind of numerical experiment. This can be used to calculate a wide variety of useful quantities. Major applications include:
        * Transmittance and Reflectance Spectra — by Fourier-transforming the response to a short pulse, a single simulation can yield the scattering amplitudes over a broadband spectrum.
        * Resonant Modes and Frequencies — by analyzing the response of the system to a short pulse, one can extract the frequencies, decay rates, and field patterns of the harmonic modes of lossy and lossless systems including waveguide and cavity modes.
        * Field Patterns (e.g. Green's functions) — in response to an arbitrary source via a continuous-wave (CW) input (fixed-ω)."
    
    Meep's scriptable interface makes it possible to combine many sorts of computations along with multi-parameter optimization in sequence or in parallel." [2]


# Introduction to the Python interface

Mostly follow Python syntax for objects, functions, etc.

![maxwell](images/python.PNG)

Image rom https://www.w3schools.com/python/python_classes.asp

**MPB** : [ModeSolver class](https://mpb.readthedocs.io/en/latest/Python_User_Interface/#the-modesolver-class)

**MEEP** : [Simulation class](https://meep.readthedocs.io/en/latest/Python_User_Interface/#the-simulation-class)


### Neat Notebook tricks

* Call terminal : !my_commands
* Autocomplete/suggest : press tab
* Cell magics : %%capture to suppress or capture output, etc.

# References

Most of the following is extracted from the Python User Interface and Tutorial pages of :

1. https://mpb.readthedocs.io/en/latest/
2. https://meep.readthedocs.io/en/latest/

The source code (thanks open-source!) itself and examples are also a good source of information :

3. https://github.com/NanoComp/meep
4. http://www.simpetus.com/projects.html